# Creo un nuovo dataset 


---
## Analisi dei commenti di un ristorante

### Da Peppino - Torino

URL di riferimento: https://www.tripadvisor.com/Restaurant_Review-g187855-d946209-Reviews-Da_Peppino-Turin_Province_of_Turin_Piedmont.html

Home --> Ristoranti --> Torino 

In [1]:
# IMPORT NECESSARI
import requests
from bs4 import BeautifulSoup

print("Imported")

Imported


In [2]:
page_URL = "https://www.tripadvisor.it/Restaurant_Review-g187855-d946209-Reviews-Da_Peppino-Turin_Province_of_Turin_Piedmont.html"

data = requests.get(page_URL)
clear_data = BeautifulSoup(data.content, "lxml")

print("Done")

Done


---
### STRUTTURA HTML DEL SITO 

#### REVIEW
`<p class="partial_entry"> qui c'è la review </p>`

#### N° TOTALE DI REVIEWS
`<a data-page-number="25" ...>
    25
 </a>`
 
#### NAMES
`<span class="expand_inline scrname" …>AlexandraCapeTown</span>`

#### RATINGS
`<span class="ui_bubble_rating bubble_30"></span>`

---
### PRIMO SCRAPING (sulla prima pagina)

Ottengo i nomi, i commenti, le review e da dove è stato scritto il commento per quanto rigurda la prima pagina. Ciò è necessario, in quanto, dalla seconda in poi l'URL cambia, permettendoci di iterare.

In [3]:
# OTTENGO UNA LISTA DI DIV CHE CONTENGONO NOME, RECENSIONE E VOTO

all_reviews = []
all_names = []
all_ratings = []
all_mobile = []

review_container = clear_data.findAll("div", attrs={"class":"review-container"})

for container in review_container:
  
    ### REVIEW ###
    review = container.find('p', attrs={'class': 'partial_entry'})
    if review:
        all_reviews.append(review.text)
        
    ### NAME ###
    name = container.find('span', attrs={'class': ('expand_inline', 'scrname')})
    if name:
      all_names.append(name.text)
    else:
      all_names.append('NO_NAME')
    
    ### RATING ###
    rating = container.find('span', attrs={'class': 'ui_bubble_rating'})
    all_ratings.append(rating['class'][1][7:8])
    
    ### MOBILE ###
    mobile = container.find('span', attrs={'class': 'viaMobile'})
    if mobile:
      all_mobile.append('Mobile')
    else:
      all_mobile.append('PC')
    
    
################################################################################################################################
        
print("Executed")
print('Nomi: ', len(all_names))
print('Commenti: ', len(all_reviews))
print('Voti: ', len(all_ratings))
print('Scritto da: ', len(all_ratings))

Executed
Nomi:  10
Commenti:  10
Voti:  10
Scritto da:  10


### SECONDO SCRAPING (dalla seconda pagina in poi)

Notiamo dall'URL che è possibile iterare sulle pagine, ottenendo i dati!

In [4]:
from tqdm import tqdm

URL_COMPLETO = 'https://www.tripadvisor.it/Restaurant_Review-g187855-d946209-Reviews-Da_Peppino-Turin_Province_of_Turin_Piedmont.html'

URL_p1 = 'https://www.tripadvisor.it/Restaurant_Review-g187855-d946209-Reviews-or'
URL_p2 = '0-Da_Peppino-Turin_Province_of_Turin_Piedmont.html'
    
# OTTENGO I COMMENTI, NOMI E VOTI DALLA SECONDA PAGINA IN POI

total_pages = int(clear_data.select("a.pageNum.last.taLnk")[0].text) # 25 come intero

# per iterare dobbiamo partire dalla seconda pagina dei commenti, così nell'URL compaiono le cifre che ci servono
for r in tqdm(range(1,total_pages)):

    # OTTENGO L'URL DELLA PAGINA
    URL_temp = URL_p1 + str(r) + URL_p2

    # OTTENGO I DATI E CI APPLICO BEATIFULSOUP
    data = requests.get(URL_temp)
    clear_data = BeautifulSoup(data.content, "lxml")
    

    review_container = clear_data.findAll("div", attrs={"class":"review-container"})

    for container in review_container:
  
      ### REVIEW ###
      review = container.find('p', attrs={'class': 'partial_entry'})
      if review:
          all_reviews.append(review.text)

      ### NAME ###
      name = container.find('span', attrs={'class': ('expand_inline', 'scrname')})
      if name:
        all_names.append(name.text)
      else:
        all_names.append('NO_NAME')

      ### RATING ###
      rating = container.find('span', attrs={'class': 'ui_bubble_rating'})
      all_ratings.append(rating['class'][1][7:8])

      ### MOBILE ###
      mobile = container.find('span', attrs={'class': 'viaMobile'})
      if mobile:
        all_mobile.append('Mobile')
      else:
        all_mobile.append('PC')
    
        
################################################################################################################################
        
print("Executed")
print('total_pages: ', total_pages)
print('Nomi: ', len(all_names))
print('Commenti: ', len(all_reviews))
print('Voti: ', len(all_ratings))
print('Scritto da: ', len(all_mobile))

100%|██████████| 55/55 [00:59<00:00,  1.15s/it]

Executed
total_pages:  56
Nomi:  559
Commenti:  559
Voti:  559
Scritto da:  559


In [5]:
# display(all_reviews)

In [6]:
# print(all_names)

In [7]:
#print(all_ratings, '\n')
#print(len(all_ratings))

In [8]:
#display(all_mobile)

#### Problema recensioni troppo lunghe
Alla fine di alcune recensioni troviamo un '...Più', che non possiamo espandere. Data la mole di dati, decidiamo dunque di non considerarle.

In [9]:
all_reviews_filtered = []
counter_reviews_scartate = 0;
counter_reviews_da_usare = 0;

for review in all_reviews:
    if '...Più' in review:
        all_reviews_filtered.append('RECENSIONE_TROPPO_LUNGA')
        counter_reviews_scartate+=1;
    else:
        all_reviews_filtered.append(review)
        counter_reviews_da_usare+=1;
        
        
print(
    'Totale recensioni:', counter_reviews_scartate + counter_reviews_da_usare, '\n',
    'Recensioni analizzabili:', counter_reviews_da_usare, '\n',
    'Recensioni scartate:', counter_reviews_scartate, '\n')

display(all_reviews_filtered[:10])

Totale recensioni: 559 
 Recensioni analizzabili: 238 
 Recensioni scartate: 321 



["Per carità. Chi dice che è buono probabilmente non capisce nulla di cucina.\nL'ho scelto per la comodità al centro città, per poter passeggiare dopo cena. Le recensioni mi hanno ingannata. Cucina veramente scadente.",
 'RECENSIONE_TROPPO_LUNGA',
 'Ristorante nelle vie del centro di torino. Perfetto per una cena senza troppe pretese. Menu completo a meno di 10€ il dolce a 4.5€ qualcosa non torna.',
 'Lo stile è quello di una trattoria tipica, non solo torinese. ottimo rapporto qualità prezzo. Personale super gentile. Consigliato',
 'Non ci ritornerò. Cucina scarsa e di bassa qualità. Lo scontrino poi...non è fiscale e solo in contanti. Se tornerò a Torino mi guarderò bene di non tornare in questo locale.',
 'RECENSIONE_TROPPO_LUNGA',
 'Locale centrale, curato negli interni. Il personale è cordiale. Abbiamo ordinato tre menù fissi, 12€ ciascuno. Ampia scelta fra primi, secondi e contorni. Consigliato.',
 'RECENSIONE_TROPPO_LUNGA',
 'RECENSIONE_TROPPO_LUNGA',
 'Storico locale in pieno c

---
### Analisi dei nomi
Cerchiamo di capire se chi ha commentato è un uomo (M) o una donna (F).

#### Elimino spazi e numeri
Ho bisogno di tenere solo le informazioni riguardanti il nome, elimino il resto.

In [10]:
import re

all_names_filtered = []

regex = re.compile('[^a-zA-Z]')

for name in all_names:
    
    if ' ' in name:                           # se c'è uno spazio nel nome
        space = name.index(' ')               # ne trovo l'indice
        name = regex.sub('', name[:space])    # effettuo un controllo su tutto ciò che non è alfabeto fino a quell'indice
        all_names_filtered.append(name)       # e aggiungo la stringa
        
    else:
        name = regex.sub('', name)            # in caso contrario effettuo un controllo su tutto ciò che non è alfabeto
        all_names_filtered.append(name)       # e aggiungo la stringa
        
print(all_names, '\n')
print(all_names_filtered)

['datemiretta', 'alessiomariano', 'Stefano C', '_laico89_veronica', 'Massimo P', 'Marco P', 'utenz', 'Frega86', 'federicobZ1848YO', 'breddula7', 'Gretolona', 'MarilenaSt', 'alessandromX6615HV', 'cale240', 'La_sosta_di_Gulliver', 'Martinarossi19', 'AndreaH5053', 'roberto-biker', 'Gab S', 'GiuseppeG344', 'paolobN1228FO', 'lucapU1521SC', 'T7332BBmarcom', 'nunziom051', 'ucufucuvv', 'MrOrly', 'Alessandro27390', 'Toto M', 'Michele d', 'giovanna N', '455claudiad', 'Andrea B', 'affannielisa', 'Mario B', 'Alessandro C', 'Giulio B', 'GoPlaces782392', 'memipipi', 'BoardingPass774711', 'giano66', 'tiziana t', 'Alisa G', 'EmanueleN100', 'angelof555', 'fabiosax04', 'pasqualino d', 'Luca F', 'Summer_12months', 'Gianni S', 'gabrieleg841', 'ciaociao1970', 'Danieleelbarbe', 'valeriat356', 'Ren408', 'Alessandro C', 'Andrea M', 'amustoph', 'E2127VXdanieles', 'Mauro T', 'EnricoR1019', 'NO_NAME', 'NO_NAME', 'NO_NAME', 'NO_NAME', 'NO_NAME', 'NO_NAME', 'NO_NAME', 'NO_NAME', 'NO_NAME', 'NO_NAME', 'NO_NAME', 'N

#### Utilizzo gender_guesser
Infine, posso utilizzare la libreria gender_guesser.

NB: Genderize ti permette di analizzare 1000 nomi al giorno.

In [11]:
!pip install gender_guesser

In [12]:
import gender_guesser.detector as gender
d = gender.Detector()

names_with_gender = []

for name in all_names_filtered:
    temp_gender = d.get_gender(name)
    names_with_gender.append(temp_gender)
  
print(names_with_gender)

['unknown', 'unknown', 'male', 'unknown', 'male', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'andy', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'male', 'unknown', 'female', 'unknown', 'unknown', 'female', 'unknown', 'male', 'male', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'female', 'unknown', 'unknown', 'unknown', 'unknown', 'male', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'andy', 'male', 'female', 'unknown', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'female', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'male', 'unknown', 'unknown', '

In [13]:
all_genders = []
i = 0

for guess in names_with_gender:
    if guess == 'male' or guess == 'mostly_male':
        all_genders.append('M')
    elif guess == 'female' or guess == 'mostly_female':
        all_genders.append('F')
    else:
        all_genders.append('UNKNOWN')

    
print(all_genders)

['UNKNOWN', 'UNKNOWN', 'M', 'UNKNOWN', 'M', 'M', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'M', 'UNKNOWN', 'F', 'UNKNOWN', 'UNKNOWN', 'F', 'UNKNOWN', 'M', 'M', 'M', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'F', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'M', 'UNKNOWN', 'M', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'M', 'F', 'UNKNOWN', 'UNKNOWN', 'M', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'F', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'M', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'F', 'F', 'UNKNOWN', 'M', 'UNKNOWN

---
## Salviamo ora i dati ottenuti in una tabella Excel
Così non dovremo ogni volta rieseguire tutto il codice



### Controlliamo che ci siano lo stesso numero di dati
Potremo così creare una tabella sfruttando Pandas

In [14]:
print('Nomi: ', len(all_names_filtered))
print('Commenti: ', len(all_reviews_filtered))
print('Voti: ', len(all_ratings))
print('Mobile: ', len(all_mobile))

Nomi:  559
Commenti:  559
Voti:  559
Mobile:  559


### Eliminiamo le recensioni troppo lunghe e quelle di cui non sappiamo il genere del commentatore
Creo una lista di liste [ ['NOME', 'GENERE', 'RECENSIONE', 'VOTO'], ['NOME', 'GENERE', 'RECENSIONE', 'VOTO'], ... ]

In [15]:
complete_list = []

i = len(all_names_filtered)

for indice in range(i):
  if all_genders[indice] != 'UNKNOWN' and all_reviews_filtered[indice] != 'RECENSIONE_TROPPO_LUNGA':
    temp = [all_names_filtered[indice], all_genders[indice], all_reviews_filtered[indice], all_ratings[indice], all_mobile[indice]]
    complete_list.append(temp)
    
print('DATI ANALIZZABILI: ', len(complete_list))
display(complete_list[:5])


DATI ANALIZZABILI:  55


[['Stefano',
  'M',
  'Ristorante nelle vie del centro di torino. Perfetto per una cena senza troppe pretese. Menu completo a meno di 10€ il dolce a 4.5€ qualcosa non torna.',
  '3',
  'Mobile'],
 ['Massimo',
  'M',
  'Non ci ritornerò. Cucina scarsa e di bassa qualità. Lo scontrino poi...non è fiscale e solo in contanti. Se tornerò a Torino mi guarderò bene di non tornare in questo locale.',
  '1',
  'Mobile'],
 ['Andrea',
  'F',
  'Ho cenato in questo ristorante la scorsa settimana e sono rimasto molto soddisfatto. Buon rapporto qualità/prezzo. Personale simpatico, cortese e disponibile. Ci tornerò sicuramente.',
  '5',
  'Mobile'],
 ['Luca',
  'M',
  'Passavo di lì per caso e ho letto il menu e ho approfittato del menu serale a 10€, devo dire che sono rimasto sorpreso, personale gentilissimo e rapporto qualità prezzo ottimo, ci tornerò con la famiglia prossimamente',
  '5',
  'Mobile'],
 ['Andrea',
  'F',
  "Consiglio vivamente questo ristorante vista l'ottimo rapporto prezzo/qualit

### Creiamo la tabella

In [16]:
# !pip install pandas
# !pip install xlsxwriter
import pandas as pd

In [17]:
# Create a Pandas dataframe from the data.
df = pd.DataFrame({'Nomi': [elemento[0] for elemento in complete_list], 
                   'Genere': [elemento[1] for elemento in complete_list], 
                   'Recensioni': [elemento[2] for elemento in complete_list],
                   'Voti': [elemento[3] for elemento in complete_list],
                   'Scritta da': [elemento[4] for elemento in complete_list]})

# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('RISTORANTE_17.xlsx', engine='xlsxwriter')

# Convert the dataframe to an XlsxWriter Excel object.
df.to_excel(writer, sheet_name='Sheet1')

# Close the Pandas Excel writer and output the Excel file.
writer.save()

In [18]:
partial_dataframe = pd.read_excel("RISTORANTE_17.xlsx")

display(partial_dataframe)

,Nomi,Genere,Recensioni,Voti,Scritta da
0,Stefano,M,Ristorante nelle vie del centro di torino. Per...,3,Mobile
1,Massimo,M,Non ci ritornerò. Cucina scarsa e di bassa qua...,1,Mobile
2,Andrea,F,Ho cenato in questo ristorante la scorsa setti...,5,Mobile
3,Luca,M,Passavo di lì per caso e ho letto il menu e ho...,5,Mobile
4,Andrea,F,Consiglio vivamente questo ristorante vista l'...,5,Mobile
5,Raffaella,F,Tempi di attesa lunghi ma sopratutto pizza cru...,1,Mobile
6,Manrico,M,"Tutto nel titolo. La qualità è buona, servizio...",5,PC
7,Giuseppe,M,ottimo ristorante menu molto completo con tant...,5,Mobile
8,Claudio,M,Ristorante pizzeria nei pressi di piazza Caste...,5,Mobile
9,Claudio,M,Locale tradizionale (esiste dal 1960!).\nCon c...,3,Mobile
